# 5'-UTR

In [22]:
head Azucena.gff

Chr08	NAM	gene	5387535	5392729	.	-	.	ID=gene:OsAzu_08g0005870;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
Chr08	NAM	mRNA	5388142	5392670	.	-	.	ID=transcript:OsAzu_08g0005870.01;Parent=gene:OsAzu_08g0005870;biotype=protein_coding;transcript_id=OsAzu_08g0005870.01;canonical_transcript=1;Tag=NAM
Chr08	NAM	exon	5388142	5390830	.	-	.	Parent=transcript:OsAzu_08g0005870.01;Name=OsAzu_08g0005870.01.exon.1;ensembl_end_phase=-1;ensembl_phase=-1;exon_id=OsAzu_08g0005870.01.exon.1;rank=2;Tag=NAM
Chr08	NAM	exon	5392180	5392670	.	-	.	Parent=transcript:OsAzu_08g0005870.01;Name=OsAzu_08g0005870.01.exon.2;ensembl_end_phase=2;ensembl_phase=2;exon_id=OsAzu_08g0005870.01.exon.2;rank=1;Tag=NAM
Chr08	NAM	CDS	5388247	5390830	.	-	1	ID=CDS:OsAzu_08g0005870.01;Parent=transcript:OsAzu_08g0005870.01;protein_id=OsAzu_08g0005870.01;Tag=NAM
Chr08	NAM	CDS	5392180	5392670	.	-	0	ID=CDS:OsAzu_08g0005870.01;Parent=transcript:OsAzu_08g0005870.01;protein_id=OsAzu_08g0005870.01;Tag=NAM
Chr08	NAM	three_prime_UTR	5388

In [131]:
#extract all five_prime_UTR
awk -F'\t' '$3 ~ /five_prime_UTR/' Azucena.gff > azucena_5UTR.gff

In [136]:
wc -l azucena_5UTR.bed

47861 azucena_5UTR.bed


In [135]:
awk '{print $1, $4-1, $5}' azucena_5UTR.gff > azucena_5UTR.bed

In [138]:
sort -k1,1 -k2,2n -k3,3n azucena_5UTR.bed -o azucena_5UTR.bed

In [139]:
head azucena_5UTR.bed

Chr01 2325 2327
Chr01 2759 2814
Chr01 10736 11163
Chr01 12076 12139
Chr01 15657 15964
Chr01 15765 15766
Chr01 22199 22623
Chr01 22206 22597
Chr01 26494 26594
Chr01 26494 26594


In [143]:
tail azucena_5UTR.bed

chr12	27463344	27463386
chr12	27464694	27464769
chr12	27464694	27464915
chr12	27464694	27464915
chr12	27475702	27475706
chr12	27475802	27476357
chr12	27476261	27476386
chr12	27476261	27476386
chr12	27476261	27476386
chrUN-Ctg41	25208	25418


In [147]:
mv azucena_5UTR1.bed azucena_5UTR.bed

In [10]:
!head azucena_5UTR.bed

head: cannot open 'azucena_5UTR.bed' for reading: No such file or directory


In [1]:
#statistics on the genomic intervals under study
import numpy as np

def bed_stats(bed_file):
    # Initialize variables
    sizes = []
    total_size = 0

    # Read in bed file and calculate sizes
    with open(bed_file, 'r') as f:
        for line in f:
            fields = line.strip().split('\t')
            start, end = int(fields[1]), int(fields[2])
            size = end - start
            sizes.append(size)
            total_size += size

    # Calculate median, minimum, maximum, and quartile values
    n = len(sizes)
    median = sorted(sizes)[n // 2] if n % 2 == 1 else sum(sorted(sizes)[n // 2 - 1:n // 2 + 1]) / 2
    minimum = min(sizes)
    maximum = max(sizes)
    q1 = np.percentile(sizes, 25)
    q3 = np.percentile(sizes, 75)

    # Print results
    print("Median size: {}".format(median))
    print("Minimum size: {}".format(minimum))
    print("Maximum size: {}".format(maximum))
    print("First quartile: {}".format(q1))
    print("Third quartile: {}".format(q3))

In [15]:
bed_stats('/scratch/ak8725/az_mrg/azucena_5UTR.bed')

Median size: 114.0
Minimum size: 1
Maximum size: 4545
First quartile: 59.0
Third quartile: 209.0


In [9]:
cd /scratch/ak8725/az_mrg

In [16]:
!bedtools makewindows -b ../ipynb\ scripts\ and\ plots/az_chr_coord.bed -w 1000 > azucena_1kb_windows.bed

In [7]:
bedtools intersect -a azucena_200bp_windows.bed -b azucena_5UTR.bed -wao > azucena_5UTR_in_windows.bed

***** WARNING: File azucena_5UTR.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	49325	49350

***** WARNING: File azucena_5UTR.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	49325	49350



In [10]:
rm azucena_5UTR_in_windows.bed

In [11]:
mv azucena_5UTR_in_windows1.bed azucena_5UTR_in_windows.bed

In [9]:
cut -f 1-3,7 azucena_5UTR_in_windows.bed >  azucena_5UTR_in_windows1.bed

In [12]:
head azucena_5UTR_in_windows.bed

chr01	0	200	0
chr01	200	400	0
chr01	400	600	0
chr01	600	800	0
chr01	800	1000	0
chr01	1000	1200	0
chr01	1200	1400	0
chr01	1400	1600	0
chr01	1600	1800	0
chr01	1800	2000	0


In [13]:
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_5UTR_in_windows.bed > azucena_5UTR_in_windows_grouped.bed

In [14]:
awk '{ $4 = $4 / 1000; print }' azucena_5UTR_in_windows_grouped.bed > azucena_5UTR_in_windows_fr.bed

In [20]:
mv azucena_5UTR_in_windows_fr.bed azucena_5UTR_in_windows.bedgraph

In [21]:
/share/apps/kent/385/bedGraphToBigWig azucena_5UTR_in_windows.bedgraph azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_5UTR_per_200bp.bw

# promoter = 1kb upstream of 'gene' feature


In [5]:
module load bedops/intel/2.4.39

In [25]:
gff2bed < azucena_genes.gff > azucena_genes.bed

In [31]:
sed -i 's/^Chr/chr/' azucena_genes.bed

In [35]:
grep -v "chrUN" azucena_genes.bed > temp.bed && mv temp.bed azucena_genes.bed

In [39]:
head azucena_genes.bed

chr01	2325	10183	gene:OsAzu_01g0000010	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000010;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	10736	11801	gene:OsAzu_01g0000020	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000020;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	10737	11793	gene:OsAzu_01g0000030	.	-	NAM	gene	.	ID=gene:OsAzu_01g0000030;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	12076	15281	gene:OsAzu_01g0000040	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000040;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	15657	19273	gene:OsAzu_01g0000050	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000050;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	22199	26348	gene:OsAzu_01g0000060	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000060;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	26494	28061	gene:OsAzu_01g0000070	.	+	NAM	gene	.	ID=gene:OsAzu_01g0000070;biotype=protein_coding;logic_name=cshl_gene;Tag=NAM
chr01	29224	33894	gene:OsAzu_01g0000080	.	+	NAM	gene	.	ID=gene:OsAzu_01g00000

In [63]:
awk '{if ($6 == "+") print $1, $2-1000, $2; else if ($6 == "-") print $1, $3, $3+1000;}' azucena_genes.bed > azucena_1kb_promoter.bed

In [64]:
sort -k1,1 -k2,2n -k3,3n azucena_1kb_promoter.bed > temp.bed && mv temp.bed azucena_1kb_promoter.bed

In [8]:
head azucena_1kb_promoter.bed

chr01	1325	2325
chr01	9736	10736
chr01	11076	12076
chr01	11793	12793
chr01	14657	15657
chr01	21199	22199
chr01	25494	26494
chr01	28224	29224
chr01	34014	35014
chr01	55572	56572


In [10]:
mv azucena_1kb_promoterT.bed azucena_1kb_promoter.bed 

In [3]:
bed_stats('/scratch/ak8725/az_mrg/azucena_1kb_promoter.bed')

Median size: 1000
Minimum size: 1000
Maximum size: 1000


In [6]:
cd /scratch/ak8725/az_mrg

In [ ]:
#sliding window 10kb, step size 1kb

In [30]:
bedtools makewindows -b ../ipynb\ scripts\ and\ plots/az_chr_coord.bed -w 2000 > azucena_2kb_windows.bed

In [10]:
module load bedtools/intel/2.29.2

In [31]:
bedtools intersect -a azucena_2kb_windows.bed -b azucena_1kb_promoter.bed -wao > azucena_promoter_in_windows.bed

***** WARNING: File azucena_1kb_promoter.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	50143	51143

***** WARNING: File azucena_1kb_promoter.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	50143	51143



In [32]:
cut -f 1-3,7 azucena_promoter_in_windows.bed >  azucena_prom_in_windows1.bed

In [36]:
head azucena_prom_in_windows.bedgraph

chr01 0 2000 0.3375
chr01 2000 4000 0.1625
chr01 4000 6000 0
chr01 6000 8000 0
chr01 8000 10000 0.132
chr01 10000 12000 0.9335
chr01 12000 14000 0.4345
chr01 14000 16000 0.5
chr01 16000 18000 0
chr01 18000 20000 0


In [33]:
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_prom_in_windows1.bed > azucena_prom_in_windows_grouped.bed

In [34]:
awk '{ $4 = $4 / 2000; print }' azucena_prom_in_windows_grouped.bed > azucena_prom_in_windows_fr.bed

In [35]:
mv azucena_prom_in_windows_fr.bed azucena_prom_in_windows.bedgraph

In [37]:
/share/apps/kent/385/bedGraphToBigWig azucena_prom_in_windows.bedgraph azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_1kb_promoter_per_2kb.bw

In [24]:
#make tab-delimited
tr ' ' '\t' < azucena_prom_in_windows.bedgraph > azucena_prom_in_windows.bedgraphT

# TE all from Raul Castaneda

In [39]:
sed -i 's/CM020633.1/chr01/g; s/CM020634.1/chr02/g; s/CM020635.1/chr03/g; s/CM020636.1/chr04/g; s/CM020637.1/chr05/g; s/CM020638.1/chr06/g; s/CM020639.1/chr07/g; s/CM020640.1/chr08/g; s/CM020641.1/chr09/g; s/CM020642.1/chr10/g; s/CM020643.1/chr11/g; s/CM020644.1/chr12/g' /scratch/ak8725/SV-TIPs\ from\ Raúl\ Castanera/TE\ annotation/GCA_009830595.1_AzucenaRS1.fasta.out.bed

In [41]:
mv /scratch/ak8725/SV-TIPs\ from\ Raúl\ Castanera/TE\ annotation/GCA_009830595.1_AzucenaRS1.fasta.out.bed /scratch/ak8725/SV-TIPs\ from\ Raúl\ Castanera/TE\ annotation/azucena_TEs.bed

In [16]:
bed_stats('/scratch/ak8725/SV-TIPs_from_Raúl_Castanera/TE_annotation/azucena_TEs.bed')

Median size: 188
Minimum size: 11
Maximum size: 146429
First quartile: 106.0
Third quartile: 332.0


In [5]:
#find fraction of 500bp windows covered by TEs 

chr01	498	655	Os1968;DNAnona/Helitron	412	+
chr01	722	787	Os1968;DNAnona/Helitron	260	+
chr01	1594	1662	Os1072;DNAnona/Helitron	268	+
chr01	1662	1838	Os0073;DNAnona/unknown	1332	-
chr01	1901	2290	Os1667;MITE/Tourist	3263	+
chr01	5052	5252	Os2924;MITE/Stow	911	-
chr01	6313	6363	Os3295;DNAauto/MULE	258	-
chr01	8245	8506	Os0231;MITE/Stow	441	-
chr01	10384	10473	Os2076;DNAnona/MULE	461	-
chr01	10477	10638	Os0974;DNAnona/MULE	329	+


In [2]:
bedtools makewindows -b ../ipynb\ scripts\ and\ plots/az_chr_coord.bed -w 500 > azucena_500bp_windows.bed

In [5]:
module load bedtools/intel/2.29.2

In [3]:
bedtools intersect -a azucena_500bp_windows.bed -b /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/TE_annotation/azucena_TEs.bed -wao > azucena_te_in_windows.bed

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/TE_annotation/azucena_TEs.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	19422	22274	Os0850_LTR;LTR/Gypsy	12869	+

***** WARNING: File /scratch/ak8725/SV-TIPs_from_Raúl_Castanera/TE_annotation/azucena_TEs.bed has a record where naming convention (leading zero) is inconsistent with other files:
chr10	19422	22274	Os0850_LTR;LTR/Gypsy	12869	+



In [2]:
cut -f 1-3,10 azucena_te_in_windows.bed >  azucena_te_in_windows1.bed

In [3]:
head azucena_te_in_windows1.bed

chr01	0	500	2
chr01	500	1000	155
chr01	500	1000	65
chr01	1000	1500	0
chr01	1500	2000	68
chr01	1500	2000	176
chr01	1500	2000	99
chr01	2000	2500	290
chr01	2500	3000	0
chr01	3000	3500	0


In [6]:
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_te_in_windows1.bed > azucena_te_in_windows_grouped.bed

In [7]:
awk '{ $4 = $4 / 500; print }' azucena_te_in_windows_grouped.bed > azucena_te_in_windows_fr.bed

In [13]:
mv azucena_te_in_windows_fr.bed azucena_te_per_500bp.bedgraph

mv: cannot stat 'azucena_te_in_windows_fr.bed': No such file or directory


: 1

In [14]:
/share/apps/kent/385/bedGraphToBigWig azucena_te_per_500bp.bedgraph azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_TE_per_500bp.bw


# SV breakpoint density
from SNP-Seek 3K RG Large Structural Variants release 1.0

In [22]:
wget https://s3.amazonaws.com/3kricegenome/reduced/CNVnator_Q10_goodRD_noCN1-3.vcf.gz

--2023-04-21 13:04:55--  https://s3.amazonaws.com/3kricegenome/reduced/CNVnator_Q10_goodRD_noCN1-3.vcf.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.109.78, 52.217.201.240, 52.217.193.24, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.109.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1096525917 (1.0G) [application/octet-stream]
Saving to: ‘CNVnator_Q10_goodRD_noCN1-3.vcf.gz’

CNVnator_Q10_goodRD 100%[===================>]   1.02G  25.5MB/s    in 32s     

2023-04-21 13:05:28 (32.4 MB/s) - ‘CNVnator_Q10_goodRD_noCN1-3.vcf.gz’ saved [1096525917/1096525917]



In [19]:
tar -xvf NB_INV_mergesam_clustered.tar

NB_INV_mergesam_clustered.txt


In [1]:
cd /scratch/ak8725/az_mrg/SV_from_SNP_Seek

In [4]:
head NB_INV_mergesam_clustered.txt

chr01	1276	1329	INV;53	IRIS_313-11671	1
chr01	1707	1993	INV;286	CX328	2
chr01	27067	27125	INV;58	IRIS_313-11797	3
chr01	36347	36399	INV;52	B121	4
chr01	38257	38345	INV;94	IRIS_313-9970	5
chr01	43444	43541	INV;97	B015	6
chr01	43444	43541	INV;97	IRIS_313-11665	6
chr01	44410	44468	INV;58	IRIS_313-11039	7
chr01	46928	47065	INV;137	IRIS_313-11252	8
chr01	72445	72544	INV;99	IRIS_313-11371	9


In [19]:
#extract breakpoints
#for deletions only
#awk 'BEGIN {OFS="\t"} {print $1, $2, $2+1; print $1, $3-1, $3}' NB_INV_mergesam_clustered.txt > npb_del.bed

In [18]:
#awk 'BEGIN {OFS="\t"} {print $1, $2, $2+1}' NB_DUP_mergesam_clustered.txt > npb_dup.bed

In [6]:
#lifted the coordinates of breakpoints using liftOver from npb_x.bed to az_x.bed
head az_del.bed

chr01	295	296
chr01	307	308
chr01	295	296
chr01	307	308
chr01	295	296
chr01	307	308
chr01	424	425
chr01	443	444
chr01	424	425
chr01	457	458


In [2]:
cd /scratch/ak8725/az_mrg/SV_from_SNP_Seek

In [2]:
wc -l az_del.bed

76180332 az_del.bed


In [4]:
#bedtools intersect -a ../azucena_200bp_windows.bed -b ins.bed -wao > azucena_ins_in_windows.bed

In [3]:
module load bedtools/intel/2.29.2

In [2]:
sbatch intersect.sh

sbatch: error: Unable to open file intersect.sh


: 1

In [6]:
head azucena_inv_in_windows.bed

chr01	0	200	.	-1	-1	0
chr01	200	400	.	-1	-1	0
chr01	400	600	.	-1	-1	0
chr01	600	800	chr01	642	643	1
chr01	800	1000	.	-1	-1	0
chr01	1000	1200	chr01	1073	1074	1
chr01	1200	1400	.	-1	-1	0
chr01	1400	1600	.	-1	-1	0
chr01	1600	1800	.	-1	-1	0
chr01	1800	2000	.	-1	-1	0


In [7]:
#save only the number of features in window
cut -f 1-3,7 azucena_del_in_windows.bed >  azucena_del_in_windows1.bed
cut -f 1-3,7 azucena_ins_in_windows.bed >  azucena_ins_in_windows1.bed
cut -f 1-3,7 azucena_dup_in_windows.bed >  azucena_dup_in_windows1.bed
cut -f 1-3,7 azucena_inv_in_windows.bed >  azucena_inv_in_windows1.bed

In [8]:
head azucena_del_in_windows1.bed

chr01	0	200	0
chr01	200	400	1
chr01	200	400	1
chr01	200	400	1
chr01	200	400	1
chr01	200	400	1
chr01	200	400	1
chr01	400	600	1
chr01	400	600	1
chr01	400	600	1


In [9]:
#find total number of features per window
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_del_in_windows1.bed > azucena_del.bedgraph
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_ins_in_windows1.bed > azucena_ins.bedgraph
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_dup_in_windows1.bed > azucena_dup.bedgraph
bedtools groupby -g 1,2,3 -c 4 -o sum -i azucena_inv_in_windows1.bed > azucena_inv.bedgraph

In [10]:
head azucena_del.bedgraph

chr01	0	200	0
chr01	200	400	6
chr01	400	600	76
chr01	600	800	0
chr01	800	1000	0
chr01	1000	1200	0
chr01	1200	1400	0
chr01	1400	1600	0
chr01	1600	1800	0
chr01	1800	2000	2


In [11]:
#convert to bw
/share/apps/kent/385/bedGraphToBigWig azucena_del.bedgraph ../azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_del_per_200bp.bw
/share/apps/kent/385/bedGraphToBigWig azucena_ins.bedgraph ../azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_ins_per_200bp.bw
/share/apps/kent/385/bedGraphToBigWig azucena_dup.bedgraph ../azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_dup_per_200bp.bw
/share/apps/kent/385/bedGraphToBigWig azucena_inv.bedgraph ../azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_inv_per_200bp.bw


In [2]:
bed_stats('/scratch/ak8725/az_mrg/SV_from_SNP_Seek/NB_DEL_mergesam_clustered.txt')

Median size: 274.0
Minimum size: 10
Maximum size: 499524
First quartile: 22.0
Third quartile: 1894.0


In [3]:
bed_stats('/scratch/ak8725/az_mrg/SV_from_SNP_Seek/NB_INS_mergesam_clustered.txt')

Median size: 1
Minimum size: 1
Maximum size: 1
First quartile: 1.0
Third quartile: 1.0


In [4]:
bed_stats('/scratch/ak8725/az_mrg/SV_from_SNP_Seek/NB_DUP_mergesam_clustered.txt')

Median size: 5576.0
Minimum size: 18
Maximum size: 499511
First quartile: 137.0
Third quartile: 17767.0


In [5]:
bed_stats('/scratch/ak8725/az_mrg/SV_from_SNP_Seek/NB_INV_mergesam_clustered.txt')

Median size: 3881
Minimum size: 3
Maximum size: 499827
First quartile: 254.0
Third quartile: 44680.0


In [4]:
#group all SV breakpoints
bedtools unionbedg -i azucena_del.bedgraph azucena_ins.bedgraph azucena_inv.bedgraph azucena_dup.bedgraph -filler 0 -scores sum > azucena_SV.bedgraph

In [9]:
mv azucena_SVs.bedgraph azucena_SV.bedgraph

1898143 azucena_SV.bedgraph


In [6]:
awk '{sum = $4 + $5 + $6 + $7; print $1"\t"$2"\t"$3"\t"sum}' azucena_SV.bedgraph > azucena_SVs.bedgraph

In [10]:
/share/apps/kent/385/bedGraphToBigWig azucena_SV.bedgraph ../azucena.chrom.sizes_TRUE /scratch/ak8725/az_mrg/fitcons/azucena_SV_per_200bp.bw


In [ ]:
if grep -q "Osaj.11" /scratch/ak8725/az_mrg/fitcons/PhastCons8wayChrAll.1.bdg; then
  echo "String found in file."
else
  echo "String not found in file."
f